In [2]:
import numpy as np
import pandas as pd
import os

In [3]:


def color_balance(img,name):
    # 1. Calculate the average intensity of each color channel
    red_avg = np.mean(img[:,:,2])
    green_avg = np.mean(img[:,:,1])
    blue_avg = np.mean(img[:,:,0])

    # 2. Compute the difference between the average intensity and the mean intensity
    mean_intensity = np.mean([red_avg, green_avg, blue_avg])
    red_diff = mean_intensity - red_avg
    green_diff = mean_intensity - green_avg
    blue_diff = mean_intensity - blue_avg

    # 3. Adjust the intensity of each channel
    img[:,:,2] = np.clip(img[:,:,2] + red_diff, 0, 255)
    img[:,:,1] = np.clip(img[:,:,1] + green_diff, 0, 255)
    img[:,:,0] = np.clip(img[:,:,0] + blue_diff, 0, 255)
    variable = name
    filename = f'balanced_{variable}'

    
    # Get the base name of the image file without the extension
    base_name = os.path.splitext(variable)[0]
    
    # Create a directory with the base name of the image inside the "images" directory if it doesn't exist
    directory = os.path.join('images', base_name)
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # Create the filename for the balanced image
    filename = os.path.join(directory, f'balanced_{variable}')
    
    # Save the image
    cv2.imwrite(filename, img)
    return img







In [4]:
import cv2
import numpy as np

def estimate_background_light_for_each_channel(image):
    # Initialize background light for each channel
    background_light_channels = np.zeros(3)
    
    # Convert image to grayscale
    gray_image = image
    
    # Initialize threshold for quadtree division
    threshold = gray_image.size * 0.001
    
    # Define function for quadtree division
    def quadtree_division(image, threshold):
        variance = np.var(image)
        
        # If variance is less than threshold, return mean intensity as background light
        if variance < threshold:
            return np.mean(image)
        
        # Otherwise, divide image into four quadrants
        height, width = image.shape
        half_height, half_width = height // 2, width // 2
        quadrants = [
            image[:half_height, :half_width],
            image[:half_height, half_width:],
            image[half_height:, :half_width],
            image[half_height:, half_width:]
        ]
        
        # Recursively call quadtree_division on each quadrant
        background_lights = [quadtree_division(quadrant, threshold) for quadrant in quadrants]
        
        # Return the minimum background light among quadrants
        return min(background_lights)
    
    # Start quadtree division for each channel
    for i in range(3):  # Assuming BGR image
        background_light_channels[i] = quadtree_division(image[:,:,i], threshold)
    print("Estimated Background Light for Each Channel:", background_light_channels)
    return background_light_channels






In [5]:
def calculate_contrast_and_complex_performance_index(I, t):
    
    # Calculate the number of channels
    num_channels = I.shape[2]
    
    # Initialize arrays to store contrast values Ci for each channel
    Ci = np.zeros(num_channels)
    
    # Loop through each color channel
    for i in range(num_channels):
       
        # Calculate the contrast value Ci for the current channel
        squared_diff_channel = (I[:,:,i] - np.mean(I[:,:,i])) ** 2
        Ci[i] = np.sum(squared_diff_channel / (t ** 2 * I.size))
    
    # Calculate the complex contrast performance index Ec
    Ec = -np.sum(Ci)
    
    return Ci, Ec



In [6]:
def calculate_information_loss_El(I, A, t):
    # Initialize J(x) array
    J = np.zeros_like(I)
    
    # Calculate J(x) using the restoration model equation for each channel
    for i in range(3):  # Assuming RGB image

        J[:,:,i] = (1 / t) * (I[:,:,i] - A[i]) + A[i]
    
    # Ensure J is within the allowable range (0 to 255)
    J_clipped = np.clip(J, 0, 255)
    
    # Calculate information loss function El
    El = 0
    
    # Loop through each color channel
    for i in range(3):  # Assuming RGB image
        # Calculate information loss for each pixel in the channel
        loss_channel = (np.minimum(0, J_clipped[:,:,i])**2) + np.maximum(0, J_clipped[:,:,i] - 255)
        
        # Sum up the information loss for the channel
        El += np.sum(loss_channel)
    
    return El



In [8]:
from scipy.optimize import minimize





def combined_objective_function(Ec, El, lambda_L):
    # Calculate the combined objective function
    return Ec + lambda_L * El

def optimize_transmittance(I, A, lambda_L, initial_guesses):
    # Initialize lists to store results
    optimal_t = []
    optimal_Ec = []
    optimal_El = []
    optimal_E = []
    print(len(initial_guesses),"kk")
    # Loop through each initial guess for t
    for i, initial_guess in enumerate(initial_guesses):
       

       
            # Calculate contrast value Ci and complex contrast performance index Ec
        Ci, Ec = calculate_contrast_and_complex_performance_index(I, initial_guess)
    
            # Calculate information loss El
        El = calculate_information_loss_El(I, A, initial_guess)
            
            # Calculate combined objective function E
        E = combined_objective_function(Ec, El, lambda_L)
            
            # Store results
        
                
        
        
        # Store the results for this initial guess
        optimal_t.append(initial_guess)
        optimal_Ec.append(Ec)
        optimal_El.append(El)
        optimal_E.append(E)
    
    # Find the index corresponding to the minimum value of E among all initial guesses
    best_index = np.argmin(optimal_E)
    
    # Return the best results
    return optimal_t[best_index], optimal_Ec[best_index], optimal_El[best_index], optimal_E[best_index]

# Example usage:
# Read RGB image



In [10]:
import cv2
import numpy as np

def restore_image(I, A, t,name):
    # Initialize the restored image
    restored_image = np.zeros_like(I)

    # Iterate over each pixel in the image
    for i in range(I.shape[0]):
        for j in range(I.shape[1]):
            # Apply the restoration model formula for each channel
            for k in range(3):  # Iterate over R, G, B channels
                restored_image[i, j, k] = (1 / t[i, j]) * (I[i, j, k] - A[k]) + A[k]
  

    # Get the base name of the image file without the extension
    base_name = os.path.splitext(name)[0]

    # Create a directory with the base name of the image inside the "images" directory if it doesn't exist
    directory = os.path.join('images', base_name)
    

    # Create the filename for the restored image
    restored_name = os.path.join(directory, f'restored_{name}.jpg')

    # Save the image
    cv2.imwrite(restored_name, restored_image)
    return restored_image




In [11]:
import numpy as np

def calculate_average_red_channel_value(image):
    # Assuming 'image' is a 3D NumPy array representing the image (height x width x channels)
    red_channel = image[:, :, 0]  # Assuming red channel is the first channel (index 0)
    average_red_value = np.mean(red_channel)
    print("Average Red Channel Value:", average_red_value)
    return average_red_value



In [12]:

def calculate_histogram_threshold(image):
    # Calculate total number of pixels
    height, width, _ = image.shape
    total_pixels = height * width
    print(total_pixels)
    # Calculate histogram threshold (h1) as n*0.225%
    histogram_threshold = total_pixels * 0.00225
    print("Histogram Threshold:", histogram_threshold)
    return  histogram_threshold




In [14]:
import numpy as np


def histogram_stretching(image, R_ave, R_threshold,name):
    # Calculate total number of pixels
    height, width, _ = image.shape
    total_pixels = height * width
    
    # Determine if attenuation of red light is slight or heavy
    if R_ave >= R_threshold:  # Slight attenuation
        channels = [0, 1, 2]  # Stretching for all channels (R, G, B)
    else:  # Heavy attenuation
        channels = [1, 2]  # Stretching for G, B channels only
    
    # Calculate satisfactory threshold ht as n * 0.225%
    ht = total_pixels * 0.00225
    
    # Calculate minimal and maximal scalar values based on ht
    imin = 0
    imax = 255
    
    # If ht is provided and it's greater than zero, update imin and imax
    if ht > 0:
        # Calculate lower and upper thresholds based on ht
        imin = np.min(image)
        imax = np.max(image)
        lower_threshold_count = int(ht)
        upper_threshold_count = int(total_pixels - ht)
        sorted_pixels = np.sort(image.flatten())
        imin = sorted_pixels[lower_threshold_count]
        imax = sorted_pixels[upper_threshold_count]

    # Apply histogram stretching to selected channels
    stretched_image = image.copy()
    for channel in channels:
        for i in range(height):
            for j in range(width):
                old_value = stretched_image[i, j, channel]
                if old_value < imin:
                    stretched_image[i, j, channel] = 0
                elif old_value > imax:
                    stretched_image[i, j, channel] = 255
                else:
                    stretched_image[i, j, channel] = 255 * (old_value - imin) / (imax - imin)
    base_name = os.path.splitext(name)[0]

    # Create a directory with the base name of the image inside the "images" directory if it doesn't exist
    directory = os.path.join('images', base_name)
    

    # Create the filename for the restored image
    stretched_name = os.path.join(directory, f'strectched_{name}.jpg')
    cv2.imwrite(stretched_name, stretched_image)
    return stretched_image






In [15]:
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage import color

def calculate_rms_contrast(image):
    """
    Calculate the Root Mean Square (RMS) contrast of an image.
    """
    return np.sqrt(np.mean(np.square(image - np.mean(image))))

def calculate_uciqe(image):
    """
    Calculate the Underwater Color Image Quality Evaluation (UCIQE) of an image.
    """
    c1 = 0.4680
    c2 = 0.2745
    c3 = 0.2576
    img_yuv = color.rgb2yuv(image)
    y = img_yuv[:,:,0]
    u = img_yuv[:,:,1]
    v = img_yuv[:,:,2]
    uciqe = c1 * np.std(y) / np.mean(y) + c2 * np.cov(u, v) / (np.std(u) * np.std(v)) + c3 * np.mean(v)
    return uciqe

def calculate_psnr(image_true, image_test):
    """
    Calculate the Peak Signal-to-Noise Ratio (PSNR) between two images.
    """
    return psnr(image_true, image_test)



In [17]:
import cv2
import numpy as np
import pandas as pd

def process_image(name):
    img = cv2.imread(name)
    # Apply color balance
    balanced_img = color_balance(img,name)
    # Estimate background light for each channel
    background_light_channels = estimate_background_light_for_each_channel(balanced_img)
    # Background light A obtained from Step 1 (assuming some value)
    A = background_light_channels
    # Tradeoff factor
    lambda_L = 5
    # Initial guesses for transmittance
    constant = 0.1  # or any small value you prefer
    initial_guesses = [np.around(np.clip(np.random.rand(*balanced_img.shape[:2]) + constant, constant,1), decimals=2) for _ in range(10)]
    # Optimize transmittance
    t_optimal, Ec_optimal, El_optimal, E_optimal = optimize_transmittance(balanced_img, A, lambda_L,initial_guesses )
    # Restore the image using the restoration model and the calculated transmittance map
    restored_image = restore_image(balanced_img, A, t_optimal,name)
    average_red = calculate_average_red_channel_value(restored_image)
    histogram_threshold = calculate_histogram_threshold(restored_image)
    r,_,_ = cv2.split(restored_image)
    retval, thresholded_image = cv2.threshold(r, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    average_thresholded = np.mean(thresholded_image)
    print(average_thresholded,"d")
    # Apply histogram stretching
    stretched_image = histogram_stretching(restored_image, average_red, average_thresholded,name)
    # Calculate metrics
    rms_contrast1 = calculate_rms_contrast(img)
    rms_contrast2 = calculate_rms_contrast(stretched_image)
    uciqe1 = calculate_uciqe(img)
    uciqe2 = calculate_uciqe(stretched_image)
    psnr_value = calculate_psnr(img, stretched_image)
    return rms_contrast1, rms_contrast2, uciqe1, uciqe2, psnr_value



In [18]:
# List of image names
image_names = ["1.jpg", "2.jpg",  "6.jpg", "7.jpg", "8.jpg",  "10.jpg"]

# Function to process image and return a tuple with all metrics
def process_and_collect_metrics(name):
    rms_contrast1, rms_contrast2, uciqe1, uciqe2, psnr_value = process_image(name)
    return (name, rms_contrast1, rms_contrast2, uciqe1, uciqe2, psnr_value)

# Process the images sequentially
results = [process_and_collect_metrics(name) for name in image_names]

# Unpack the results into separate lists
image_names, rms_contrast1_list, rms_contrast2_list, uciqe1_list, uciqe2_list, psnr_list = zip(*results)

# Create a DataFrame for each metric
rms_contrast_df = pd.DataFrame({'Image': image_names, 'RMS Contrast (Base Image)': rms_contrast1_list, 'RMS Contrast (Final Image)': rms_contrast2_list})
uciqe_df = pd.DataFrame({'Image': image_names, 'UCIQE (Base Image)': uciqe1_list, 'UCIQE (Final Image)': uciqe2_list})
psnr_df = pd.DataFrame({'Image': image_names, 'PSNR': psnr_list})

# Print the DataFrames

Estimated Background Light for Each Channel: [107.205025   107.27166667 107.2474    ]
10 kk
Average Red Channel Value: 109.53251666666667
120000
Histogram Threshold: 270.0
65.605125 d


C:\Users\gauth\AppData\Local\Temp\ipykernel_21204\2771436528.py:44: RuntimeWarning: overflow encountered in scalar multiply
  stretched_image[i, j, channel] = 255 * (old_value - imin) / (imax - imin)


Estimated Background Light for Each Channel: [ 85.07306667  55.10105263 134.01956667]
10 kk
Average Red Channel Value: 135.9047
120000
Histogram Threshold: 270.0
131.206 d
Estimated Background Light for Each Channel: [ 74.1044      72.51533333 124.14475833]
10 kk
Average Red Channel Value: 132.31994166666667
120000
Histogram Threshold: 270.0
127.12175 d
Estimated Background Light for Each Channel: [123.09353333 105.51933333 122.89369167]
10 kk
Average Red Channel Value: 124.84715
120000
Histogram Threshold: 270.0
129.758875 d
Estimated Background Light for Each Channel: [123.69449167  72.60631579 123.66061667]
10 kk
Average Red Channel Value: 127.32836666666667
120000
Histogram Threshold: 270.0
167.42875 d
Estimated Background Light for Each Channel: [71.07339167  7.44444444 71.18971667]
10 kk
Average Red Channel Value: 84.68436666666666
120000
Histogram Threshold: 270.0
35.7255 d


In [19]:
rms_contrast_df.to_csv('rms_contrast.csv', index=False)
rms_contrast_df


,Image,RMS Contrast (Base Image),RMS Contrast (Final Image)
0,1.jpg,15.576318,120.515799
1,2.jpg,31.119433,119.608896
2,6.jpg,28.549699,119.842003
3,7.jpg,17.034188,102.610918
4,8.jpg,18.686693,103.457288
5,10.jpg,19.715753,119.432462


In [20]:
uciqe_df.to_csv('uciqe.csv', index=False)

uciqe_df


,Image,UCIQE (Base Image),UCIQE (Final Image)
0,1.jpg,"[[0.2453441707007674, 0.2453507556232119, 0.24...","[[0.4605544839086259, 0.36793135517462433, 0.3..."
1,2.jpg,"[[0.18026506839471965, 0.18025123930132497, 0....","[[0.5444059141048395, 0.3782670983148085, 0.35..."
2,6.jpg,"[[0.22053915655401252, 0.22028431915748525, 0....","[[0.6600979150116887, 0.4415526042754434, 0.42..."
3,7.jpg,"[[0.10995485439610392, 0.1100073298032756, 0.1...","[[0.42144673892276274, 0.29835719919420234, 0...."
4,8.jpg,"[[0.11843055261436622, 0.11850060036200667, 0....","[[0.439485912698537, 0.364098885811428, 0.3584..."
5,10.jpg,"[[1.7087114912624133, 1.70746820176356, 1.6730...","[[0.48657368956298275, 0.37087970630790046, 0...."


In [21]:
psnr_df.to_csv('psnr.csv', index=False)
psnr_df

,Image,PSNR
0,1.jpg,6.046188
1,2.jpg,6.593471
2,6.jpg,6.524777
3,7.jpg,8.118390
4,8.jpg,8.110475
5,10.jpg,4.738160
